In [1]:
from __future__ import absolute_import, division, print_function
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.text import text_to_word_sequence,one_hot,Tokenizer
from keras.preprocessing import sequence
import numpy as np
import gensim
from gensim.models import Word2Vec
import pandas as pd
import multiprocessing
import csv

Using TensorFlow backend.
C:\Users\anoth\Miniconda3\envs\tensorflow\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
# Load Google's pre-trained Word2Vec model.
model = gensim.models.KeyedVectors.load_word2vec_format('./GoogleNews-vectors-negative300.bin', binary=True,limit=450000)

In [3]:
data=[]
label=[]
with open('train.tsv') as tsvfile:
  reader = csv.reader(tsvfile, delimiter='\t')
  for row in reader:
        r=[]
        if len(row[2].split())<=25 and row[3]!='2':
                for i in (row[2].split()):
                    try:
                        r.append(model[i])
                    except KeyError:
                        r.append(np.zeros(300))
                k=len(r)
                if k<25:
                        for i in range(0,25-k):
                            r.append(np.zeros(300))
                if (row[3]=='0' or row[3]=='1'):
                        label.append(0)
                elif (row[3]=='3' or row[3]=='4'):
                        label.append(1)
                data.append(r)
del model
del data[0]                
traindata=np.array(data[:10000])
testdata=np.array(data[10000:20000])
del data
trainlabel=np.array(label[:10000])
testlabel=np.array(label[10000:20000])
del label
print(len(traindata))
print(len(trainlabel))

10000
10000


In [4]:
def cnnmodel(features, labels, mode):    
    input_layer = tf.reshape(features["x"], [-1, 25,300,1])
    conv1 = tf.layers.conv2d(inputs=input_layer,filters=50,kernel_size=[4, 5],strides=5,activation=tf.nn.relu)
    print(conv1.shape)
    pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)
    print(pool1.shape)
    conv2 = tf.layers.conv2d(inputs=pool1,filters=100,kernel_size=[5, 6],padding="same",activation=tf.nn.relu)
    print(conv2.shape)
    pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)
    print(pool2.shape)
    conv3 = tf.layers.conv2d(inputs=pool2,filters=200,kernel_size=[6, 7],padding="same",activation=tf.nn.relu)
    print(conv3.shape)
    pool3 = tf.layers.max_pooling2d(inputs=conv3, pool_size=[1,1], strides=2)
    print(pool3.shape)
    pool3_flat= tf.reshape(pool3,[-1,1*8*200])
    logits = tf.layers.dense(inputs=pool3_flat, units=2, activation=tf.nn.softmax)
    
    predictions = {
      "classes": tf.argmax(input=logits, axis=1),
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
      }
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)
    
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)
    
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
        train_op = optimizer.minimize(
            loss=loss,
            global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)
    
    eval_metric_ops = {
          "accuracy": tf.metrics.accuracy(
              labels=labels, predictions=predictions["classes"])
      }
    return tf.estimator.EstimatorSpec( mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

In [5]:
analyzer = tf.estimator.Estimator(model_fn=cnnmodel, model_dir="/tmp/sentimentanalysis")
tensors_to_log = {"probabilities": "softmax_tensor"}
logging_hook = tf.train.LoggingTensorHook(tensors=tensors_to_log, every_n_iter=50)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/sentimentanalysis', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000018ACAD42898>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [8]:
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": traindata},
    y=trainlabel,
    batch_size=10000,
    num_epochs=None,
    shuffle=True)
analyzer.train(
    input_fn=train_input_fn,
    steps=10,
    hooks=[logging_hook]
)

INFO:tensorflow:Calling model_fn.
(10000, 5, 60, 50)
(10000, 2, 30, 50)
(10000, 2, 30, 100)
(10000, 1, 15, 100)
(10000, 1, 15, 200)
(10000, 1, 8, 200)
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/sentimentanalysis\model.ckpt-12
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 12 into /tmp/sentimentanalysis\model.ckpt.
INFO:tensorflow:probabilities = [[0.50080805 0.49919195]
 [0.50026514 0.49973486]
 [0.50069666 0.49930334]
 ...
 [0.50015769 0.49984231]
 [0.50042884 0.49957116]
 [0.5000636  0.4999364 ]]
INFO:tensorflow:loss = 0.6932361721992493, step = 13
INFO:tensorflow:Saving checkpoints for 16 into /tmp/sentimentanalysis\model.ckpt.
Instructions for updating:
Use standard file APIs to delete files with this prefix.
INFO:tensorflow:Saving checkpoints for 20 into /tmp/sentimentanalysis\model.ck

In [9]:
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": testdata},
    y=testlabel,
    num_epochs=1,
    shuffle=False)

eval_results = analyzer.evaluate(input_fn=eval_input_fn)
print(eval_results)

INFO:tensorflow:Calling model_fn.
(?, 5, 60, 50)
(?, 2, 30, 50)
(?, 2, 30, 100)
(?, 1, 15, 100)
(?, 1, 15, 200)
(?, 1, 8, 200)
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-04-27T23:47:59Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/sentimentanalysis\model.ckpt-22
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-04-27-23:48:39
INFO:tensorflow:Saving dict for global step 22: accuracy = 0.4659, global_step = 22, loss = 0.6932046
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 22: /tmp/sentimentanalysis\model.ckpt-22
{'accuracy': 0.4659, 'loss': 0.6932046, 'global_step': 22}
